## Installation of TwelveLabs SDKs

In [1]:
!pip install -U -q twelvelabs

## Configure your API key
Add API Key as a env variable. Signup for TwelveLabs and Get your API keys [here](https://playground.twelvelabs.io/?_gl=1*1rx2bxa*_ga*MTUzMTIzMTI3MC4xNzI3OTAxMzE2*_ga_END0TB2RFD*MTc0MDE4NTM2NS4yNC4xLjE3NDAxODY1MjkuMC4wLjA.). No credit card is required to use the Free plan. The Free plan includes indexing of 600 mins of videos, which is enough for a small project.

### Create an AWS Secrets Manager secret 

Follow steps here to [create a secret](https://docs.aws.amazon.com/secretsmanager/latest/userguide/create_secret.html) in AWS Secrets Manager. For example, name the secret as `TL_API_Key`.

Note down the ARN or name of the secret (`TL_API_Key`) to retrieve. To retrieve a secret from another account, you must use an ARN.

For an ARN, we recommend that you specify a complete ARN rather than a partial ARN. See [Finding a secret from a partial ARN](https://docs.aws.amazon.com/secretsmanager/latest/userguide/troubleshoot.html#ARN_secretnamehyphen).

Use the name of the secret for the `SecretId` in the code block below.

In [2]:
import boto3
import json
secrets_manager_client=boto3.client("secretsmanager")
API_secret=secrets_manager_client.get_secret_value(
    SecretId="TL_API_KEY"
)
TL_API_KEY=json.loads(API_secret["SecretString"])["TL_API_Key"]

## Using video data

For the demo, you will on these videos: [Robin bird forest Video by Federico Maderno from Pixabay](https://pixabay.com/videos/robin-bird-forest-nature-spring-21723/) and [Island Video by Bellergy RC from Pixabay](https://pixabay.com/videos/island-motorboat-sea-ocean-2946/). 

This demo depends upon some video data. To use this, we will download 2 mp4 files and upload it to an Amazon S3 bucket.

1. Click on the links containing the [Robin bird forest Video by Federico Maderno from Pixabay](https://pixabay.com/videos/robin-bird-forest-nature-spring-21723/) and [Island Video by Bellergy RC from Pixabay](https://pixabay.com/videos/island-motorboat-sea-ocean-2946/) videos.
1. Download the 21723-320725678_small.mp4 and 2946-164933125_small.mp4 files.
1. Create an S3 bucket if you don't have one already. Follow the steps in the [Creating a bucket doc](https://docs.aws.amazon.com/AmazonS3/latest/userguide/create-bucket-overview.html). Note down the name of the bucket and replace it the code block below (Eg., `MYS3BUCKET`).
1. Upload the 21723-320725678_small.mp4 and 2946-164933125_small.mp4 video files to the S3 bucket created in the step above by following the stesp in the [Uploading objects doc](https://docs.aws.amazon.com/AmazonS3/latest/userguide/upload-objects.html). Note down the name of the objects and replace it the code block below (Eg., `21723-320725678_small.mp4` and `2946-164933125_small.mp4`)


In [3]:
s3_client=boto3.client("s3")
bird_video_data=s3_client.download_file(Bucket='MYS3BUCKET',  Key='21723-320725678_small.mp4', Filename='robin-bird.mp4')
island_video_data=s3_client.download_file(Bucket='MYS3BUCKET',  Key='2946-164933125_small.mp4', Filename='island.mp4')

## Generate Embeddings
Use the Embed API to create multimodal embeddings that are contextual vector representations for your videos and texts. Twelve Labs video embeddings capture all the subtle cues and interactions between different modalities, including the visual expressions, body language, spoken words, and the overall context of the video, encapsulating the essence of all these modalities and their interrelations over time.

To create video embeddings, you must first upload your videos, and the platform must finish processing them. Uploading and processing videos require some time. Consequently, creating embeddings is an asynchronous process comprised of three steps:

1. Upload and process a video: When you start uploading a video, the platform creates a video embedding task and returns its unique task identifier.

2. Monitor the status of your video embedding task: Use the unique identifier of your task to check its status periodically until it's completed.

3. Retrieve the embeddings: After the video embedding task is completed, retrieve the video embeddings by providing the task identifier.
Learn more in the [docs](https://docs.twelvelabs.io/docs/create-video-embeddings)

In [4]:
from twelvelabs import TwelveLabs
from typing import List
from twelvelabs.models.embed import EmbeddingsTask, SegmentEmbedding

def print_segments(segments: List[SegmentEmbedding], max_elements: int = 1024):
    for segment in segments:
        print(
            f"  embedding_scope={segment.embedding_scope} start_offset_sec={segment.start_offset_sec} end_offset_sec={segment.end_offset_sec}"
        )
        print(f"  embeddings: {segment.embeddings_float[:max_elements]}")

# Initialize client with API key
twelvelabs_client = TwelveLabs(api_key=TL_API_KEY)

video_files=["robin-bird.mp4", "island.mp4"]
tasks=[]

for video in video_files:
    # Create embedding task
    task = twelvelabs_client.embed.task.create(
        model_name="Marengo-retrieval-2.7",
        video_file=video
    )
    print(
        f"Created task: id={task.id} engine_name={task.model_name} status={task.status}"
    )
    
    def on_task_update(task: EmbeddingsTask):
        print(f"  Status={task.status}")
    
    status = task.wait_for_done(
        sleep_interval=2,
        callback=on_task_update
    )
    print(f"Embedding done: {status}")
    
    # Retrieve and inspect results
    task = task.retrieve()
    if task.video_embedding is not None and task.video_embedding.segments is not None:
        print_segments(task.video_embedding.segments)
    tasks.append(task)


Created task: id=67edc5869cffa5179927b0a9 engine_name=Marengo-retrieval-2.7 status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=processing
  Status=ready
Embedding done: ready
  embedding_scope=clip start_offset_sec=0.0 end_offset_sec=6.0
  embeddings: [0.022429451, 0.00040668788, -0.01825908, -0.005862708, -0.03371106, -6.357456e-05, -0.015320076, -0.042556215, -0.02782445, -0.00019097517, 0.03258314, -0.0061399476, -0.00049206393, 0.035632476, 0.028209884, 0.02875258, -0.035486065, -0.11288028, -0.040782217, -0.0359422, 0.015908664, -0.021092793, 0.016303983, 0.06351931, -0.03370151, -0.059598114, 0.034242813, 0.013847672, -0.0077595757, 0.02485134, -0.00448969, 0.016366754, -0.015583887, 0.040098537, 0.009583407, -0.0028939361, 0.019334463, 0.028072173

## Create OpenSearch Domain

If you have an OpenSearch Service domain already, you can use the connection details for that domain. If you want to create a new domain, follow the steps in the [docs](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/createupdatedomains.html).

Follow [Operational best practices for Amazon OpenSearch Service](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/bp.html)

In [5]:
!pip install opensearch-py
!pip install botocore
!pip install requests-aws4auth

## Getting OpenSearch Domain URL
Update your domain endpoint in the `host` section below.

You can find the domain endpoint under 'General Information' within details of domain created earlier.

Example: https://search-mydomain-1a2a3a4a5a6a7a8a9a0a9a8a7a.us-east-1.es.amazonaws.com

[Create an AWS Secrets Manager secret](https://docs.aws.amazon.com/secretsmanager/latest/userguide/create_secret.html) for the Amazon OpenSearch username and password.

Note down the ARN or name of the secret (Eg., `AOS_password`, `AOS_username`) to retrieve . To retrieve a secret from another account, you must use an ARN.

For an ARN, we recommend that you specify a complete ARN rather than a partial ARN. See [Finding a secret from a partial ARN](https://docs.aws.amazon.com/secretsmanager/latest/userguide/troubleshoot.html#ARN_secretnamehyphen).

In [6]:
import json
from opensearchpy import OpenSearch, RequestsHttpConnection, helpers
from requests.auth import HTTPBasicAuth

# OpenSearch connection configuration
# host = 'your-host.aos.us-east-1.on.aws'
host = 'your-host.aos.us-east-1.on.aws'
port = 443  # Default HTTPS port

# Get OpenSearch credentials from Secrets Manager
secrets_manager_client = boto3.client('secretsmanager')

opensearch_username = secrets_manager_client.get_secret_value(
    SecretId="AOS_username"
)
opensearch_username_string = json.loads(opensearch_username["SecretString"])["AOS_username"]

opensearch_password = secrets_manager_client.get_secret_value(
    SecretId="AOS_password"
)
opensearch_password_string = json.loads(opensearch_password["SecretString"])["AOS_password"]

auth = (opensearch_username_string, opensearch_password_string)

# Create the client configuration
client_aos = OpenSearch(
    hosts=[{'host': host, 'port': port}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
)

# Test the connection
try:
    cluster_info = client_aos.info()
    print("Successfully connected to OpenSearch")
    print(f"Cluster info: {cluster_info}")
except Exception as e:
    print(f"Connection failed: {str(e)}")

# Define the enhanced index configuration
index_name = 'twelvelabs_index'
new_vector_index_definition = {
    "settings": {
        "index": {
            "knn": "true",
            "number_of_shards": 1,
            "number_of_replicas": 0
        }
    },
    "mappings": {
        "properties": {
            "embedding_field": {
                "type": "knn_vector",
                "dimension": 1024
            },
            "video_title": {
                "type": "text",
                "fields": {
                    "keyword": {
                        "type": "keyword"
                    }
                }
            },
            "segment_start": {
                "type": "date"
            },
            "segment_end": {
                "type": "date"
            },
            "segment_id": {
                "type": "text"
            }
        }
    }
}

# Create index if it doesn't exist
client_aos.indices.create(index=index_name, body=new_vector_index_definition, ignore=400)

# Function to perform vector search
def search_similar_segments(query_vector, k=5):
    query = {
        "size": k,
        "_source": ["video_title", "segment_start", "segment_end", "segment_id"],
        "query": {
            "knn": {
                "embedding_field": {
                    "vector": query_vector,
                    "k": k
                }
            }
        }
    }
    
    response = client_aos.search(
        index=index_name,
        body=query
    )

    results = []
    for hit in response['hits']['hits']:
        result = {
            'score': hit['_score'],
            'title': hit['_source']['video_title'],
            'start_time': hit['_source']['segment_start'],
            'end_time': hit['_source']['segment_end'],
            'segment_id': hit['_source']['segment_id']
        }
        results.append(result)

    return (results)

# Function to format search results
def print_search_results(results):
    print("\nSearch Results:")
    print("-" * 50)
    for i, result in enumerate(results, 1):
        print(f"\nResult {i}:")
        print(f"Video: {result['title']}")
        print(f"Time Range: {result['start_time']} - {result['end_time']}")
        print(f"Similarity Score: {result['score']:.4f}")

# Example usage
if __name__ == "__main__":
    # Check if index exists
    if client_aos.indices.exists(index=index_name):
        print(f"Index '{index_name}' configuration:")
        print(json.dumps(client_aos.indices.get(index=index_name), indent=2))
    else:
        print(f"Index '{index_name}' does not exist")


Successfully connected to OpenSearch
Cluster info: {'name': 'b5491a2c11eb609007ead1e8e17ccd49', 'cluster_name': '329430715989:twelvelabs', 'cluster_uuid': '_ZlBgMR5Rm6qJAtgJX3yMA', 'version': {'distribution': 'opensearch', 'number': '2.15.0', 'build_type': 'tar', 'build_hash': 'unknown', 'build_date': '2024-10-23T17:37:00.962018205Z', 'build_snapshot': False, 'lucene_version': '9.10.0', 'minimum_wire_compatibility_version': '7.10.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'The OpenSearch Project: https://opensearch.org/'}
Index 'twelvelabs_index' configuration:
{
  "twelvelabs_index": {
    "aliases": {},
    "mappings": {
      "properties": {
        "embedding_field": {
          "type": "knn_vector",
          "dimension": 1024
        },
        "segment_end": {
          "type": "date"
        },
        "segment_id": {
          "type": "text"
        },
        "segment_start": {
          "type": "date"
        },
        "video_title": {
          "type":

## Upsert the video embeddings into OpenSearch

In [7]:
from opensearchpy.helpers import bulk

def generate_actions(tasks, video_files):
    count = 0
    for task in tasks:
        # Check if video embeddings are available
        if task.video_embedding is not None and task.video_embedding.segments is not None:
            embeddings_doc = task.video_embedding.segments
            
            # Generate actions for bulk indexing
            for doc_id, elt in enumerate(embeddings_doc):
                yield {
                    '_index': index_name,
                    '_id': doc_id,
                    '_source': {
                        'embedding_field': elt.embeddings_float,
                        'video_title': video_files[count],
                        'segment_start': elt.start_offset_sec,
                        'segment_end': elt.end_offset_sec,
                        'segment_id': doc_id
                    }
                }
        print(f"Prepared bulk indexing data for task {count}")
        count += 1

# Perform bulk indexing
try:
    success, failed = bulk(client_aos, generate_actions(tasks, video_files))
    print(f"Successfully indexed {success} documents")
    if failed:
        print(f"Failed to index {len(failed)} documents")
except Exception as e:
    print(f"Error during bulk indexing: {e}")

Prepared bulk indexing data for task 0
Prepared bulk indexing data for task 1
Successfully indexed 6 documents


## Create Text Embeddings and Perform Text-to-Video Search

In [8]:
def print_segments(segments: List[SegmentEmbedding], max_elements: int = 1024):
    for segment in segments:
        print(
            f"  embedding_scope={segment.embedding_scope} start_offset_sec={segment.start_offset_sec} end_offset_sec={segment.end_offset_sec}"
        )
        print(f"  embeddings: {segment.embeddings_float[:max_elements]}")

# Create text embeddings
text_res = twelvelabs_client.embed.create(
  model_name="Marengo-retrieval-2.7",
  text="Bird eating food",
)

print("Created a text embedding")
print(f" Model: {text_res.model_name}")

if text_res.text_embedding is not None and text_res.text_embedding.segments is not None:
        print_segments(text_res.text_embedding.segments)

Created a text embedding
 Model: Marengo-retrieval-2.7
  embedding_scope=None start_offset_sec=None end_offset_sec=None
  embeddings: [0.036621094, -0.0061950684, 0.018188477, -0.041992188, -0.024658203, 0.018432617, -0.01953125, 0.007507324, -0.032714844, -0.017700195, 0.014465332, -0.020019531, 0.007659912, 0.018676758, 0.029785156, -0.01965332, 0.0010299683, -0.022827148, 0.0022277832, -0.055908203, 0.040771484, 0.0032806396, -0.01928711, 0.05126953, -0.030883789, -0.030395508, 0.033447266, 0.029296875, 0.009277344, 0.021118164, -0.016479492, 0.006378174, -0.006591797, -0.007537842, 0.0045166016, -0.006378174, 0.033203125, 0.018554688, -0.010986328, -0.017456055, -0.03930664, -0.018798828, 0.045898438, 0.026489258, -0.038085938, -0.007751465, -0.04321289, 0.03930664, -0.02368164, -0.00044631958, 0.00074386597, -0.029418945, 0.007537842, 0.029663086, 0.028198242, 0.017822266, -0.014099121, -0.04663086, 0.030395508, 0.007019043, 0.049560547, -0.029663086, 0.01953125, -0.004638672, 0.0

In [9]:
vector_search = text_res.text_embedding.segments[0].embeddings_float
print (vector_search)

[0.036621094, -0.0061950684, 0.018188477, -0.041992188, -0.024658203, 0.018432617, -0.01953125, 0.007507324, -0.032714844, -0.017700195, 0.014465332, -0.020019531, 0.007659912, 0.018676758, 0.029785156, -0.01965332, 0.0010299683, -0.022827148, 0.0022277832, -0.055908203, 0.040771484, 0.0032806396, -0.01928711, 0.05126953, -0.030883789, -0.030395508, 0.033447266, 0.029296875, 0.009277344, 0.021118164, -0.016479492, 0.006378174, -0.006591797, -0.007537842, 0.0045166016, -0.006378174, 0.033203125, 0.018554688, -0.010986328, -0.017456055, -0.03930664, -0.018798828, 0.045898438, 0.026489258, -0.038085938, -0.007751465, -0.04321289, 0.03930664, -0.02368164, -0.00044631958, 0.00074386597, -0.029418945, 0.007537842, 0.029663086, 0.028198242, 0.017822266, -0.014099121, -0.04663086, 0.030395508, 0.007019043, 0.049560547, -0.029663086, 0.01953125, -0.004638672, 0.009643555, 0.013671875, -0.012023926, -0.033203125, 0.04248047, 0.043701172, 0.018188477, -0.0034484863, 0.033447266, 0.04248047, 0.054

In [10]:
# Perform vector search
query_vector = vector_search
text_to_video_search = search_similar_segments(query_vector)
# print(text_video_search)
    
print_search_results(text_to_video_search)


Search Results:
--------------------------------------------------

Result 1:
Video: robin-bird.mp4
Time Range: 18.0 - 21.087732
Similarity Score: 0.4409

Result 2:
Video: robin-bird.mp4
Time Range: 12.0 - 18.0
Similarity Score: 0.4300

Result 3:
Video: island.mp4
Time Range: 0.0 - 6.0
Similarity Score: 0.3624


## Create Audio Embeddings and Perform Audio-to-Video Search

We will work on this audio data to showcase audio to video search: [Rock 808 beat.mp3 by T_roy_920 -- https://freesound.org/s/425556/ -- License: Creative Commons 0](https://freesound.org/people/T_roy_920/sounds/425556/). We have downloaded the audio locally and uploaded it to the S3 bucket created earlier.

In [11]:
audio_data=s3_client.download_file(Bucket='krao-test-one',  Key='425556__t_roy_920__rock-808-beat.mp3', Filename='audio-data.mp3')

In [12]:
# Create audio embeddings
audio_res = twelvelabs_client.embed.create(
    model_name="Marengo-retrieval-2.7",
    audio_file="audio-data.mp3",
)

print(f"Created audio embedding: model_name={audio_res.model_name}")
print(f" Model: {audio_res.model_name}")

if audio_res.audio_embedding is not None and audio_res.audio_embedding.segments is not None:
    print_segments(audio_res.audio_embedding.segments)

Created audio embedding: model_name=Marengo-retrieval-2.7
 Model: Marengo-retrieval-2.7
  embedding_scope=None start_offset_sec=0.0 end_offset_sec=None
  embeddings: [-0.064453125, -0.016601562, -0.0056762695, -0.014770508, 0.012207031, -0.009216309, 0.021484375, -0.015258789, -0.0039978027, -0.0039978027, 0.030639648, -0.036376953, 0.024291992, 0.0018692017, 0.012878418, 0.041015625, 0.020019531, 0.026855469, -0.007507324, -0.021728516, -0.011108398, -0.003189087, 0.032470703, -0.007446289, 0.03149414, -0.0054626465, 0.009399414, 0.025878906, 0.034423828, -0.033935547, 0.023071289, 0.027832031, -0.016723633, -0.028564453, -0.04663086, -0.044189453, -0.0154418945, -0.015136719, 0.03466797, -0.016845703, -0.020141602, -0.01550293, -0.04345703, 0.0073547363, -0.055419922, 0.021362305, 0.055664062, 0.009643555, 0.013183594, -0.0073242188, -4.172325e-06, -0.0015411377, 0.02734375, 0.0010223389, -0.026000977, -0.008911133, -0.0050354004, -0.030151367, -0.0016403198, -0.0021362305, -0.039794

In [13]:
vector_search = audio_res.audio_embedding.segments[0].embeddings_float
print (vector_search)

[-0.064453125, -0.016601562, -0.0056762695, -0.014770508, 0.012207031, -0.009216309, 0.021484375, -0.015258789, -0.0039978027, -0.0039978027, 0.030639648, -0.036376953, 0.024291992, 0.0018692017, 0.012878418, 0.041015625, 0.020019531, 0.026855469, -0.007507324, -0.021728516, -0.011108398, -0.003189087, 0.032470703, -0.007446289, 0.03149414, -0.0054626465, 0.009399414, 0.025878906, 0.034423828, -0.033935547, 0.023071289, 0.027832031, -0.016723633, -0.028564453, -0.04663086, -0.044189453, -0.0154418945, -0.015136719, 0.03466797, -0.016845703, -0.020141602, -0.01550293, -0.04345703, 0.0073547363, -0.055419922, 0.021362305, 0.055664062, 0.009643555, 0.013183594, -0.0073242188, -4.172325e-06, -0.0015411377, 0.02734375, 0.0010223389, -0.026000977, -0.008911133, -0.0050354004, -0.030151367, -0.0016403198, -0.0021362305, -0.039794922, 0.0070495605, 0.021362305, 0.030883789, 0.011657715, 0.018188477, -0.020629883, 0.0025634766, 0.03564453, 0.0067749023, -0.024902344, -0.0043029785, -0.003814697

In [14]:
# Perform vector search
query_vector = vector_search
audio_to_video_search = search_similar_segments(query_vector)
# print(text_video_search)
    
print_search_results(audio_to_video_search)


Search Results:
--------------------------------------------------

Result 1:
Video: island.mp4
Time Range: 6.0 - 12.0
Similarity Score: 0.2855

Result 2:
Video: robin-bird.mp4
Time Range: 18.0 - 21.087732
Similarity Score: 0.2841

Result 3:
Video: robin-bird.mp4
Time Range: 12.0 - 18.0
Similarity Score: 0.2837

Result 4:
Video: island.mp4
Time Range: 0.0 - 6.0
Similarity Score: 0.2835


## Create Image Embeddings and Perform Image-to-Video Search

You will work on this image data to showcase image to video search: [Ocean Image by shotbyjagar from Pixabay](https://pixabay.com/photos/ocean-water-sea-waves-seascape-5294270/). Download the audio locally and uploaded it to the S3 bucket created earlier.

In [ ]:
# Create image embeddings
image_data=s3_client.download_file(Bucket='krao-test-one',  Key='ocean-5294270_1280.jpg', Filename='image-data.jpg')

image_res = twelvelabs_client.embed.create(
    model_name="Marengo-retrieval-2.7",
    image_file="image-data.jpg",
)

print(f"Created image embedding: model_name={image_res.model_name}")
print(f" Model: {image_res.model_name}")

if image_res.image_embedding is not None and image_res.image_embedding.segments is not None:
    print_segments(image_res.image_embedding.segments)

Created image embedding: model_name=Marengo-retrieval-2.7
 Model: Marengo-retrieval-2.7
  embedding_scope=None start_offset_sec=None end_offset_sec=None
  embeddings: [0.012821175, -0.040612154, 0.006895007, 0.04964901, 2.7436665e-05, -0.03239206, -0.011105242, 0.024255488, 0.04060654, -0.011139354, -0.022674311, 0.01945592, -0.0010499081, 0.0106038, 0.02355325, 3.0775595e-06, -0.008602296, -0.086988784, 0.00207274, 0.0047872267, 0.049068224, 0.011751694, -0.012202721, 0.055041537, -0.016249876, 0.0042409, 0.009964569, -0.00742697, 0.029936176, 0.0048729014, -0.03186175, 0.008081443, -0.036665298, -0.0109246485, -0.0022452923, 0.017308904, 0.013233205, -0.003783449, -0.0005386526, -0.019298509, -0.035418198, -0.025993127, 0.0015262096, -0.004058616, 0.016318802, 0.038388405, -0.029291991, -0.01121083, -0.007107105, -0.019674828, -0.0032510632, -0.010387522, 0.044599365, 0.02672533, -0.035838526, -0.045326468, -0.035109576, 0.018327128, 0.01529006, 0.021195496, -0.0046474105, 0.01475984

In [16]:
vector_search = image_res.image_embedding.segments[0].embeddings_float
print (vector_search)

[0.012821175, -0.040612154, 0.006895007, 0.04964901, 2.7436665e-05, -0.03239206, -0.011105242, 0.024255488, 0.04060654, -0.011139354, -0.022674311, 0.01945592, -0.0010499081, 0.0106038, 0.02355325, 3.0775595e-06, -0.008602296, -0.086988784, 0.00207274, 0.0047872267, 0.049068224, 0.011751694, -0.012202721, 0.055041537, -0.016249876, 0.0042409, 0.009964569, -0.00742697, 0.029936176, 0.0048729014, -0.03186175, 0.008081443, -0.036665298, -0.0109246485, -0.0022452923, 0.017308904, 0.013233205, -0.003783449, -0.0005386526, -0.019298509, -0.035418198, -0.025993127, 0.0015262096, -0.004058616, 0.016318802, 0.038388405, -0.029291991, -0.01121083, -0.007107105, -0.019674828, -0.0032510632, -0.010387522, 0.044599365, 0.02672533, -0.035838526, -0.045326468, -0.035109576, 0.018327128, 0.01529006, 0.021195496, -0.0046474105, 0.014759848, 0.009106953, 0.095728256, -0.02122103, 0.00063410134, 0.02581465, 0.01874696, 0.03329541, 0.025144985, 0.023938017, -0.021369247, 0.0023450535, -0.0040658447, -0.01

In [17]:
# Perform vector search
query_vector = vector_search
image_to_video_search = search_similar_segments(query_vector)
# print(text_video_search)
    
print_search_results(image_to_video_search)


Search Results:
--------------------------------------------------

Result 1:
Video: island.mp4
Time Range: 6.0 - 12.0
Similarity Score: 0.5616

Result 2:
Video: island.mp4
Time Range: 0.0 - 6.0
Similarity Score: 0.5576

Result 3:
Video: robin-bird.mp4
Time Range: 12.0 - 18.0
Similarity Score: 0.4592

Result 4:
Video: robin-bird.mp4
Time Range: 18.0 - 21.087732
Similarity Score: 0.4540
